# Data Preprocessing

## Feature Selection

### Feature Selection using domain knowledge and business understanding

### Remove redundant features

Remove low variance features

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)
high_variance_features = selector.fit_transform(scaled_data)

## Preprocessing

### Handle Missing values

### Remove duplicates (if any)

### Outliers Handling

Using IQR methods

### Normalization

Min-Max scaling

### Feature Encoding

Binary label encoding

One-hot encoding

### Feature Engineering